# Multiprocessed Arbitrage Order Generation Notebook

This notebook makes multiprocessed orders using the find_arbitrage function in arbitrage.py

In [7]:
import multiprocessing as mp
from datetime import datetime
import pandas as pd
from arbitrage import find_arbitrage 

start_date: datetime = datetime(2024, 1, 1)
end_date: datetime = datetime(2024, 3, 30)

current_date = start_date
arbitrage_cutoff = 0.75

manager = mp.Manager()
orders = manager.list()

options: pd.DataFrame = pd.read_csv("data/cleaned_options_data.csv")
options["day"] = options["ts_recv"].apply(lambda x: x.split("T")[0])
options['day'] = options['day'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d"))


def process_instruments(instrument_ids, current_data):
    with mp.Pool(mp.cpu_count()) as pool:
        instrument_data_list = [current_data.loc[current_data['instrument_id'] == instrument_id, :] for instrument_id in instrument_ids]
        results = pool.map(find_arbitrage, instrument_data_list)
        
    for result in results:
        orders.extend(result)


while current_date <= end_date:
    current_data = options.loc[options['day'] == current_date, :]
    print(current_date, current_data.shape[0])
    if current_data.shape[0] == 0:
        current_date = current_date + pd.DateOffset(days=1)
        continue

    current_data = current_data.sort_values(['instrument_id', 'bid_px_00'])
    instrument_ids = current_data['instrument_id'].unique()

    process_instruments(instrument_ids, current_data)

    current_date = current_date + pd.DateOffset(days=1)

orders = list(orders)
df = pd.DataFrame(orders)
df

2024-01-01 00:00:00 0
2024-01-02 00:00:00 22556
2024-01-03 00:00:00 21409
2024-01-04 00:00:00 16526
2024-01-05 00:00:00 20568
2024-01-06 00:00:00 0
2024-01-07 00:00:00 0
2024-01-08 00:00:00 24570
2024-01-09 00:00:00 21654
2024-01-10 00:00:00 21136
2024-01-11 00:00:00 21267
2024-01-12 00:00:00 26472
2024-01-13 00:00:00 0
2024-01-14 00:00:00 0
2024-01-15 00:00:00 0
2024-01-16 00:00:00 28235
2024-01-17 00:00:00 33837
2024-01-18 00:00:00 76227
2024-01-19 00:00:00 30587
2024-01-20 00:00:00 0
2024-01-21 00:00:00 0
2024-01-22 00:00:00 33193
2024-01-23 00:00:00 21540
2024-01-24 00:00:00 29408
2024-01-25 00:00:00 20749
2024-01-26 00:00:00 18689
2024-01-27 00:00:00 0
2024-01-28 00:00:00 0
2024-01-29 00:00:00 22688
2024-01-30 00:00:00 22602
2024-01-31 00:00:00 22457
2024-02-01 00:00:00 19617
2024-02-02 00:00:00 31585
2024-02-03 00:00:00 0
2024-02-04 00:00:00 0
2024-02-05 00:00:00 21382
2024-02-06 00:00:00 16891
2024-02-07 00:00:00 27586
2024-02-08 00:00:00 26153
2024-02-09 00:00:00 30997
2024-02-

,datetime,option_symbol,action,order_size
0,2024-01-02T20:51:55.295389518Z,SPX 241220C04000000,S,37
1,2024-01-02T20:47:38.971421974Z,SPX 241220C04000000,B,37
2,2024-01-02T20:08:54.236189805Z,SPX 240621P04300000,S,50
3,2024-01-02T18:49:34.268285863Z,SPX 240621P04300000,B,50
4,2024-01-02T16:55:16.811967114Z,SPX 240621C04600000,S,50
...,...,...,...,...
47527,2024-03-28T15:26:20.441821542Z,SPX 241220P05275000,B,21
47528,2024-03-28T20:03:48.235720941Z,SPX 240920P05375000,S,30
47529,2024-03-28T19:34:38.343814675Z,SPX 240920P05375000,B,30
47530,2024-03-28T18:16:18.328707852Z,SPX 240920P05260000,S,47


In [5]:
orders = list(orders)
orders

[{'datetime': '2024-01-02T20:51:55.295389518Z',
  'option_symbol': 'SPX   241220C04000000',
  'action': 'S',
  'order_size': np.int64(37)},
 {'datetime': '2024-01-02T20:47:38.971421974Z',
  'option_symbol': 'SPX   241220C04000000',
  'action': 'B',
  'order_size': np.int64(37)},
 {'datetime': '2024-01-02T20:08:54.236189805Z',
  'option_symbol': 'SPX   240621P04300000',
  'action': 'S',
  'order_size': 50},
 {'datetime': '2024-01-02T18:49:34.268285863Z',
  'option_symbol': 'SPX   240621P04300000',
  'action': 'B',
  'order_size': 50},
 {'datetime': '2024-01-02T16:55:16.811967114Z',
  'option_symbol': 'SPX   240621C04600000',
  'action': 'S',
  'order_size': 50},
 {'datetime': '2024-01-02T15:37:49.400985492Z',
  'option_symbol': 'SPX   240621C04600000',
  'action': 'B',
  'order_size': 50},
 {'datetime': '2024-01-02T16:03:34.286244905Z',
  'option_symbol': 'SPX   240621C04200000',
  'action': 'S',
  'order_size': np.int64(48)},
 {'datetime': '2024-01-02T15:38:28.567791219Z',
  'option_sy

In [6]:
df = pd.DataFrame(orders)
df

,datetime,option_symbol,action,order_size
0,2024-01-02T20:51:55.295389518Z,SPX 241220C04000000,S,37
1,2024-01-02T20:47:38.971421974Z,SPX 241220C04000000,B,37
2,2024-01-02T20:08:54.236189805Z,SPX 240621P04300000,S,50
3,2024-01-02T18:49:34.268285863Z,SPX 240621P04300000,B,50
4,2024-01-02T16:55:16.811967114Z,SPX 240621C04600000,S,50
...,...,...,...,...
47527,2024-03-28T15:26:20.441821542Z,SPX 241220P05275000,B,21
47528,2024-03-28T20:03:48.235720941Z,SPX 240920P05375000,S,30
47529,2024-03-28T19:34:38.343814675Z,SPX 240920P05375000,B,30
47530,2024-03-28T18:16:18.328707852Z,SPX 240920P05260000,S,47


In [8]:
df

,datetime,option_symbol,action,order_size
0,2024-01-02T20:51:55.295389518Z,SPX 241220C04000000,S,37
1,2024-01-02T20:47:38.971421974Z,SPX 241220C04000000,B,37
2,2024-01-02T20:08:54.236189805Z,SPX 240621P04300000,S,50
3,2024-01-02T18:49:34.268285863Z,SPX 240621P04300000,B,50
4,2024-01-02T16:55:16.811967114Z,SPX 240621C04600000,S,50
...,...,...,...,...
47527,2024-03-28T15:26:20.441821542Z,SPX 241220P05275000,B,21
47528,2024-03-28T20:03:48.235720941Z,SPX 240920P05375000,S,30
47529,2024-03-28T19:34:38.343814675Z,SPX 240920P05375000,B,30
47530,2024-03-28T18:16:18.328707852Z,SPX 240920P05260000,S,47


In [9]:
df['prod'] = df['bid_px_00'] * df['ask_px_00']
df.sort_values('prod', ascending=False)

KeyError: 'bid_px_00'

In [11]:
df = pd.read_csv("data/cleaned_options_data.csv")
df['prod'] = df['bid_px_00'] * df['bid_sz_00']
df.sort_values('prod', ascending=False)

,ts_recv,instrument_id,bid_px_00,ask_px_00,bid_sz_00,ask_sz_00,symbol,prod
1330881,2024-03-14T13:50:45.560670410Z,1191195563,3513.80,3536.50,184,10,SPX 241220P09000000,646539.20
30645,2024-01-03T16:07:10.931919566Z,587236195,4509.00,4516.20,132,58,SPX 240216C00200000,595188.00
1462585,2024-03-20T18:42:40.747865419Z,1174429611,4987.20,5003.00,108,93,SPX 240517C00200000,538617.60
1440742,2024-03-19T16:23:20.329682296Z,1191201019,494.10,507.10,1077,1077,SPX 240920C04850000,532145.70
1318660,2024-03-13T18:55:40.932235138Z,1191191408,4945.60,4960.30,105,95,SPX 240920C00200000,519288.00
...,...,...,...,...,...,...,...,...
441995,2024-01-24T19:36:02.947234589Z,587238434,0.05,0.10,1,4153,SPX 240216C06100000,0.05
1384460,2024-03-14T19:47:33.631320636Z,1174426135,0.05,0.15,1,283,SPX 240315P04180000,0.05
881703,2024-02-15T20:49:31.357579602Z,1191200761,0.05,0.10,1,295,SPX 240216P04325000,0.05
1381636,2024-03-14T19:36:41.026026029Z,1174428562,0.05,0.10,1,611,SPX 240315C05285000,0.05
